## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-13-04-26 +0000,wapo,Ukraine adopts new anti-corruption law as prot...,https://www.washingtonpost.com/world/2025/07/3...
1,2025-07-31-13-00-37 +0000,wapo,Trump’s tariffs face major legal challenge in ...,https://www.washingtonpost.com/business/2025/0...
2,2025-07-31-13-00-04 +0000,nyt,Bess Wohl’s ‘Liberation’ Is Heading to Broadwa...,https://www.nytimes.com/2025/07/31/theater/bes...
3,2025-07-31-13-00-00 +0000,wsj,Fox Takes Stake in IndyCar Owner Penske Entert...,https://www.wsj.com/business/media/fox-takes-s...
4,2025-07-31-12-59-07 +0000,wapo,"Eight dead in Kyiv, including 6-year-old, afte...",https://www.washingtonpost.com/world/2025/07/3...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,71
11,tariffs,21
45,trade,20
63,gaza,19
2,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
199,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,156
324,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,142
16,2025-07-31-12-28-38 +0000,wapo,Trump puts trade deal in doubt after Canada ba...,https://www.washingtonpost.com/politics/2025/0...,140
57,2025-07-31-10-33-13 +0000,cbc,Trump says U.S. will set 15% tariff on South K...,https://www.cbc.ca/news/world/trump-tariffs-so...,140
189,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,138


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
199,156,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
268,87,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
361,69,2025-07-30-13-51-11 +0000,cbc,Bank of Canada holds interest rate at 2.75% as...,https://www.cbc.ca/news/business/boc-july-deci...
203,56,2025-07-30-22-31-15 +0000,nypost,Witkoff could visit Gaza aid sites during Isra...,https://nypost.com/2025/07/30/world-news/witko...
251,51,2025-07-30-20-52-58 +0000,wapo,Fed keeps rates steady but warns about slowing...,https://www.washingtonpost.com/business/2025/0...
176,47,2025-07-30-23-37-48 +0000,wapo,NTSB hearing: Army helicopter’s location syste...,https://www.washingtonpost.com/business/2025/0...
316,46,2025-07-30-17-01-29 +0000,wapo,NYC mass killing raises new security concerns ...,https://www.washingtonpost.com/business/2025/0...
309,44,2025-07-30-17-43-00 +0000,wsj,Tsunami Waves Reach U.S. After Powerful Russia...,https://www.wsj.com/world/tsunami-watch-warnin...
328,44,2025-07-30-16-23-00 +0000,wsj,Two top Senate Republicans laid out a plan Wed...,https://www.wsj.com/politics/policy/us-weapons...
119,42,2025-07-31-05-39-41 +0000,nyt,"Pro-Palestinian Group Can Appeal U.K. Ban, Jud...",https://www.nytimes.com/2025/07/30/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
